In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis2 as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

/Users/edasmalchi/miniconda3/envs/calitp/lib/python3.10/site-packages/google/cloud/bigquery/client.py:535: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


### Generating Analysis from RT Data

* Currently supporting analysis for an entire day from a single operator at a time

In [2]:
rt.OperatorDayAnalysis?

Init signature: rt.OperatorDayAnalysis(itp_id, analysis_date)
Docstring:     
New top-level class for rt delay/speed analysis of a single operator on a single day
    
File:           ~/calitp/data-analyses/rt_delay/rt_analysis2.py
Type:           type
Subclasses:     


In [3]:
analysis_date = dt.date(2022, 1, 12)

In [4]:
lbt = rt.OperatorDayAnalysis(170, analysis_date)

found parquet
found parquet
found parquet
found parquet
found_parquet
projecting
done
9289275
sc_called
check location data for trip -4329841926259451152
sc_called
(39, 12)
9288197
sc_called
check location data for trip 477904054299874101
sc_called
(70, 12)
check location data for trip 477904054299874101
sc_called
(65, 12)
check location data for trip 477904054299874101
sc_called
(63, 12)
9288671
sc_called
9288672
sc_called
check location data for trip -4332510122068877721
sc_called
(73, 12)
9288276
sc_called
check location data for trip -8407187321077433825
sc_called
(47, 12)
check location data for trip -8407187321077433825
sc_called
(43, 12)
9288469
sc_called
check location data for trip -9129792514909367336
sc_called
(59, 12)
9288189
sc_called
check location data for trip 8605761159091182331
sc_called
(76, 12)
check location data for trip 8605761159091182331
sc_called
(71, 12)
check location data for trip 8605761159091182331
sc_called
(68, 12)
check location data for trip 860576115

/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


could not generate delays for trip 9296640
cannot convert float infinity to integer
insufficient bounding points for trip 9144216795643991396, location 0.0: start/end of route?
insufficient bounding points for trip 9144216795643991396, location 17657.564072333073: start/end of route?
insufficient bounding points for trip 5693659803407109653, location 0.0: start/end of route?
insufficient bounding points for trip 5693659803407109653, location 17657.564072333073: start/end of route?
insufficient bounding points for trip 151817190631042841, location 10786.441996207377: start/end of route?
insufficient bounding points for trip 151817190631042841, location 0.0: start/end of route?
insufficient bounding points for trip 8287077767684215723, location 2056.9488362650873: start/end of route?
insufficient bounding points for trip 8287077767684215723, location 1616.843127544651: start/end of route?
insufficient bounding points for trip 8287077767684215723, location 1882.571276700251: start/end of 

/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


could not generate delays for trip 9288711
cannot convert float infinity to integer
insufficient bounding points for trip 7252661040443367171, location 31823.96151545443: start/end of route?
insufficient bounding points for trip 7252661040443367171, location 578.1490798157977: start/end of route?
insufficient bounding points for trip 7252661040443367171, location 446.22065349351: start/end of route?
insufficient bounding points for trip 7252661040443367171, location 0.00015330631844669122: start/end of route?
insufficient bounding points for trip 1920040843798764188, location 31823.96151545443: start/end of route?
insufficient bounding points for trip 1920040843798764188, location 578.1490798157977: start/end of route?
insufficient bounding points for trip 1920040843798764188, location 1045.7476703777263: start/end of route?
insufficient bounding points for trip 1920040843798764188, location 446.22065349351: start/end of route?
insufficient bounding points for trip 1920040843798764188,

...

insufficient bounding points for trip 4061315843584739662, location 6673.514016307208: start/end of route?
insufficient bounding points for trip -2213072546571975403, location 0.0: start/end of route?
insufficient bounding points for trip -2213072546571975403, location 6673.514016307208: start/end of route?
insufficient bounding points for trip 5287414126437595364, location 0.0: start/end of route?
insufficient bounding points for trip 5287414126437595364, location 6673.514016307208: start/end of route?
insufficient bounding points for trip 616487927603905251, location 0.0: start/end of route?
insufficient bounding points for trip 616487927603905251, location 456.35160059317707: start/end of route?
insufficient bounding points for trip 616487927603905251, location 6673.514016307208: start/end of route?
insufficient bounding points for trip -4498120236993229230, location 0.0: start/end of route?
insufficient bounding points for trip -4498120236993229230, location 6673.514016307208: star

/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


: start/end of route?
insufficient bounding points for trip -2888223792599402585, location 659.0581277796192: start/end of route?
insufficient bounding points for trip 3756600844047331316, location 0.0: start/end of route?
insufficient bounding points for trip 3756600844047331316, location 206.91314215097245: start/end of route?
insufficient bounding points for trip 8869078695623371549, location 0.0: start/end of route?
insufficient bounding points for trip 8869078695623371549, location 206.91314215097245: start/end of route?
insufficient bounding points for trip -8432862664072021235, location 1501.5303687710693: start/end of route?
insufficient bounding points for trip -8432862664072021235, location 0.0: start/end of route?
insufficient bounding points for trip -8432862664072021235, location 930.3294415426872: start/end of route?
insufficient bounding points for trip -8432862664072021235, location 206.91314215097245: start/end of route?
insufficient bounding points for trip -843286266

/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


insufficient bounding points for trip 6978398524389426859, location 501.0407611454837: start/end of route?
insufficient bounding points for trip 6978398524389426859, location 0.0: start/end of route?
insufficient bounding points for trip 5906203065431207719, location 0.0: start/end of route?
insufficient bounding points for trip -7074131750388588809, location 501.0407611454837: start/end of route?
insufficient bounding points for trip -7074131750388588809, location 991.1092912035456: start/end of route?
insufficient bounding points for trip -7074131750388588809, location 0.0: start/end of route?
insufficient bounding points for trip 3082123411846948636, location 501.0407611454837: start/end of route?
insufficient bounding points for trip 3082123411846948636, location 0.0: start/end of route?
insufficient bounding points for trip 3082123411846948636, location 11480.641185357437: start/end of route?
insufficient bounding points for trip -5374575435373645071, location 0.0: start/end of ro

In [5]:
marin = rt.OperatorDayAnalysis(194, analysis_date)

found parquet
found parquet
found parquet
found parquet
found_parquet
projecting
done
71049
sc_called
check location data for trip -7794768601654416563
sc_called
(739, 12)
check location data for trip -7794768601654416563
sc_called
(559, 12)
check location data for trip -7794768601654416563
sc_called
(464, 12)
check location data for trip -7794768601654416563
sc_called
(401, 12)
check location data for trip -7794768601654416563
sc_called
(352, 12)
check location data for trip -7794768601654416563
sc_called
(316, 12)
check location data for trip -7794768601654416563
sc_called
(286, 12)
check location data for trip -7794768601654416563
sc_called
(263, 12)
check location data for trip -7794768601654416563
sc_called
(244, 12)
check location data for trip -7794768601654416563
sc_called
(230, 12)
check location data for trip -7794768601654416563
sc_called
(217, 12)
check location data for trip -7794768601654416563
sc_called
(206, 12)
check location data for trip -7794768601654416563
sc_calle

/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


could not generate delays for trip 233023
cannot convert float infinity to integer
insufficient bounding points for trip 1024710949707389680, location 9176.208546925489: start/end of route?
insufficient bounding points for trip 1024710949707389680, location 0.0: start/end of route?
insufficient bounding points for trip 1265962211644618532, location 9176.208546925489: start/end of route?
insufficient bounding points for trip 1265962211644618532, location 0.0: start/end of route?
insufficient bounding points for trip -4485455801622027053, location 9176.208546925489: start/end of route?
insufficient bounding points for trip -4485455801622027053, location 0.0: start/end of route?
insufficient bounding points for trip 4811332209213790324, location 9176.208546925489: start/end of route?
insufficient bounding points for trip 4811332209213790324, location 0.0: start/end of route?
insufficient bounding points for trip 6787671774322269261, location 9176.208546925489: start/end of route?
insuffic

/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


could not generate delays for trip 233017
cannot convert float infinity to integer
insufficient bounding points for trip -1423700143228633330, location 9176.208546925489: start/end of route?
insufficient bounding points for trip -1423700143228633330, location 0.0: start/end of route?
insufficient bounding points for trip -1517564634139653046, location 9176.208546925489: start/end of route?
insufficient bounding points for trip -1517564634139653046, location 0.0: start/end of route?
insufficient bounding points for trip 5724640071535626141, location 25453.164797437363: start/end of route?
insufficient bounding points for trip 5724640071535626141, location 0.0: start/end of route?
insufficient bounding points for trip 8335699325768985165, location 25453.164797437363: start/end of route?
insufficient bounding points for trip 8335699325768985165, location 0.0: start/end of route?
insufficient bounding points for trip 1301876553805024488, location 25453.164797437363: start/end of route?
ins

...

: start/end of route?
insufficient bounding points for trip 580131159190754619, location 18953.67933881611: start/end of route?
insufficient bounding points for trip 580131159190754619, location 0.0: start/end of route?
insufficient bounding points for trip -2040995701087430109, location 0.0: start/end of route?
insufficient bounding points for trip -2040995701087430109, location 8642.696954218798: start/end of route?
insufficient bounding points for trip -5730624410624147738, location 0.0: start/end of route?
insufficient bounding points for trip -5730624410624147738, location 8642.696954218798: start/end of route?
insufficient bounding points for trip -5625855914132313924, location 0.0: start/end of route?
insufficient bounding points for trip 5199237056136090637, location 0.0: start/end of route?
insufficient bounding points for trip -116706406753249633, location 0.0: start/end of route?
insufficient bounding points for trip 7659799421003075416, location 0.0: start/end of route?
ins

/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last
/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


: start/end of route?
insufficient bounding points for trip 7534789186204048719, location 0.0: start/end of route?
insufficient bounding points for trip -7981618577145509576, location 16315.498745483244: start/end of route?
could not generate delays for trip 36004
cannot convert float infinity to integer
insufficient bounding points for trip -2445348690635266761, location 3164.9086640458495: start/end of route?
insufficient bounding points for trip -2445348690635266761, location 15918.352882074367: start/end of route?
insufficient bounding points for trip -2445348690635266761, location 16315.498745483244: start/end of route?
insufficient bounding points for trip -2445348690635266761, location 0.0: start/end of route?
insufficient bounding points for trip 8069913661305113739, location 16315.498745483244: start/end of route?
insufficient bounding points for trip 8069913661305113739, location 0.0: start/end of route?
insufficient bounding points for trip -8165043951824465904, location 163

/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:76: RuntimeWarning: divide by zero encountered in double_scalars
  est_seconds_to_next = meters_position_to_next / speed_from_last


insufficient bounding points for trip 3557621533263068593, location 16315.498745483244: start/end of route?
insufficient bounding points for trip 3557621533263068593, location 0.0: start/end of route?
insufficient bounding points for trip 3670484815816906559, location 16315.498745483244: start/end of route?
insufficient bounding points for trip 3670484815816906559, location 0.0: start/end of route?
insufficient bounding points for trip -4933645559476768076, location 16315.498745483244: start/end of route?
insufficient bounding points for trip -4933645559476768076, location 0.0: start/end of route?
insufficient bounding points for trip 4761737712078576353, location 16315.498745483244: start/end of route?
insufficient bounding points for trip 4761737712078576353, location 0.0: start/end of route?
insufficient bounding points for trip 8146537297954001197, location 16315.498745483244: start/end of route?
insufficient bounding points for trip 8146537297954001197, location 0.0: start/end of 

### Working from a Single Trip

* Each analysis has a position_interpolators attribute, a nested dict of trip_ids with an rt (and eventually a schedule) interpolator for each
* rt interpolator is the foundation of RT analysis, based on Vehicle Positions
    * trip information
    * ability to interpolate arrival time at arbitrary distances along the route (shape)
    * can generate a detailled single-trip speed map with full resolution from Vehicle Positions

In [29]:
lbt.position_interpolators['9288261']['rt'].time_at_position(5000) ## distance down the shape in meters

Timestamp('2022-01-12 13:47:39.950962')

In [30]:
lbt.position_interpolators['9288261']['rt'].route_id

'192'

In [31]:
lbt.position_interpolators['9288261']['rt'].trip_id

'9288261'

In [21]:
lbt.position_interpolators['9288261']['rt'].position_gdf >> head(3)

,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
0,2022-01-12 13:30:13,33.768063,-118.192120,2022-01-12 13:30:33,POINT (167501.086 -470293.515),44.451926,NaT,NaN,NaN,NaN,False,NaN
1,2022-01-12 13:31:14,33.768085,-118.192210,2022-01-12 13:31:33,POINT (167492.702 -470291.235),52.795329,2022-01-12 13:30:13,44.451926,61.0,8.343404,True,0.136777
2,2022-01-12 13:32:14,33.768852,-118.193726,2022-01-12 13:32:33,POINT (167350.641 -470208.871),266.678687,2022-01-12 13:31:14,52.795329,60.0,213.883357,True,3.564723


In [22]:
lbt.position_interpolators['9288261']['rt'].progressing_positions >> head(3)

,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
66675,2022-01-12 13:31:14,33.768085,-118.192210,2022-01-12 13:31:33,POINT (167492.702 -470291.235),52.795329,2022-01-12 13:30:13,44.451926,61.0,8.343404,True,0.136777
66676,2022-01-12 13:32:14,33.768852,-118.193726,2022-01-12 13:32:33,POINT (167350.641 -470208.871),266.678687,2022-01-12 13:31:14,52.795329,60.0,213.883357,True,3.564723
66677,2022-01-12 13:33:14,33.770370,-118.193680,2022-01-12 13:33:33,POINT (167351.694 -470040.483),435.011217,2022-01-12 13:32:14,266.678687,60.0,168.332530,True,2.805542


In [32]:
lbt.position_interpolators['9288261']['rt'].detailed_speed_map()

/Users/edasmalchi/miniconda3/envs/calitp/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/Users/edasmalchi/calitp/data-analyses/rt_delay/rt_analysis2.py:98: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.dissolve().centroid


### Working from an entire Operator x Day

* Includes view for delay at each stop, for all trips with RT data
* Also basic info on how many scheduled trips covered by RT

In [33]:
lbt.stop_delay_view >> head(3)

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,direction_id,actual_time,delay
0,1329,BROADWAY & WISCONSIN SE,POINT (170331.380 -470464.154),1110082,3319.806387,-5721502980648177876,9288975,12,2022-01-12 11:48:34,111,1,2022-01-12 11:48:21.142052,0 days
1,1367,LAKEWOOD & CENTRALIA NE,POINT (171955.580 -462251.798),1110082,17292.340107,-5721502980648177876,9288975,47,2022-01-12 12:28:42,111,1,2022-01-12 12:28:00.969967,0 days
2,1330,BROADWAY & TEMPLE SE,POINT (170602.891 -470529.426),1110082,3599.026438,-5721502980648177876,9288975,13,2022-01-12 11:49:20,111,1,2022-01-12 11:48:57.729418,0 days


In [34]:
(lbt.stop_delay_view
 >> group_by(_.stop_id, _.stop_name)
 >> summarize(mean_delay = _.delay.mean(), max_delay = _.delay.max())
 >> arrange(-_.mean_delay)
 >> head(3)
)

,stop_id,stop_name,mean_delay,max_delay
127,0149,COWLES & SANTA FE SW,0 days 00:33:32.411184,0 days 00:34:21.196251
1502,1996,LOS CERRITOS CENTER LAYOVER,0 days 00:17:58.926773170,0 days 00:38:54.258742
1819,4135,ARTESIA STATION BAY 3,0 days 00:11:05.832202164,0 days 00:30:12.565320


In [35]:
lbt.scheduled_trip_rt_coverage ## Percentage of scheduled trips with at least some RT data

0.9982142857142857

In [36]:
lbt.route_coverage_summary() >> head(3)

,trip_key,trip_id,shape_id,direction_id,min_meters,min_stop,max_meters,max_stop
0,-9217889075259924607,9289433,610106,0,625.200598,3,18562.339122,53
1,-9216178830099537732,9288701,1730122,1,446.220653,2,30983.929803,88
2,-9180052545081174409,9296544,10070,1,2381.776799,4,7748.095459,15


In [23]:
lbt.set_filter(start_time='12:00', end_time='15:00', direction_id='1')

In [24]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, 
 vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [26]:
lbt.filter

{'start_time': datetime.time(12, 0),
 'end_time': datetime.time(15, 0),
 'route_ids': None,
 'direction_id': '1',
 'direction': None}

In [44]:
# m = lbt.segment_speed_map(how='low_speeds', colorscale=colorscale)

In [27]:
marin.set_filter(start_time='12:00', end_time='15:00', direction_id='1')

In [43]:
# m2 = marin.segment_speed_map(how='low_speeds', colorscale=colorscale)

In [40]:
marin.position_interpolators['219002']['rt'].progressing_positions >> head(5)

,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp,geometry,shape_meters,last_time,last_loc,secs_from_last,meters_from_last,progressed,speed_from_last
118619,2022-01-12 06:40:03,37.875280,-122.459150,2022-01-12 06:40:07,POINT (-215997.876 -12882.750),375.002366,2022-01-12 06:39:44,345.142822,19.0,29.859544,True,1.571555
118622,2022-01-12 06:40:22,37.874280,-122.460010,2022-01-12 06:40:29,POINT (-216076.280 -12991.919),1303.580789,2022-01-12 06:40:03,375.002366,19.0,928.578424,True,48.872549
118633,2022-01-12 06:42:24,37.874805,-122.459496,2022-01-12 06:42:27,POINT (-216029.630 -12934.748),1377.330697,2022-01-12 06:41:53,1120.231716,31.0,257.098980,True,8.293515
118635,2022-01-12 06:42:45,37.875530,-122.459220,2022-01-12 06:42:47,POINT (-216003.303 -12854.808),1479.794345,2022-01-12 06:42:24,1377.330697,21.0,102.463648,True,4.879221
118638,2022-01-12 06:42:53,37.875620,-122.459390,2022-01-12 06:43:09,POINT (-216017.972 -12844.419),1497.756216,2022-01-12 06:42:45,1479.794345,8.0,17.961871,True,2.245234


In [42]:
# marin.position_interpolators['219002']['rt'].detailed_speed_map()